roadway network builder
description: create roadway features after centroid connector builder
Sijia.Wang@wsp.com
Feb 21, 2019

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, shape, LineString
from osgeo import ogr

In [2]:
#define input files
#osm drive network (edge/node), centroid connector file, node coordinates
cd = "C:/Users/wangs1/Documents/Met_Council/"
osm_drive_edge_file = cd + "centroid_connector_builder/osm_drive_edges_with_cc_new.shp"
osm_walk_edge_file = cd + "centroid_connector_builder/osm_walk_edges_with_cc_new.shp"
county_shape_file = cd + "GIS/cb_2017_us_county_5m/cb_2017_us_county_5m.shp"
mn_count_file = cd + "osm_roadway_network_builder/count_corresp/AADT_2017_Count_Loca_corresp.csv"
wi_count_file = cd + "osm_roadway_network_builder/count_corresp/TRADAS_(counts)_corresp.csv"
mn_fc_file = cd + "osm_roadway_network_builder/RC_corresp/trans_mrcc_centerlines.csv"
wi_fc_file = cd + "osm_roadway_network_builder/RC_corresp/WISLR.csv" 
fc_corrsp_file = cd + "osm_roadway_network_builder/RC_corresp/osm_drive_edges_with_fc_id.csv"

#define county name codes
county_name_dict = {'Anoka':1, 'Carver':2, 'Dakota':3, 'Hennepin':4, 'Ramsey':5, 'Scott':6, 'Washington':7, 'external':10,
                    'Chisago':11, 'Goodhue':12, 'Isanti':13, 'Le Sueur':14, 'McLeod':15, 'Pierce':16, 'Polk':17, 'Rice':18,
                    'Sherburne':19, 'Sibley':20, 'St. Croix':21, 'Wright':22}

#area corresp
area_file = cd + "GIS/shp_society_thrive_msp2040_com_des/ThriveMSP2040CommunityDesignation.shp"
area_code_dict = {23 : 4, 24 : 3, 25 : 2, 35 : 2, 36 : 1, 41 : 1, 51 : 1, 52 : 1, 53 : 1, 60 : 1}

#asgngrp corresp
mn_asgngrp_dict = {1 : 1, 2 : 2, 3 : 6, 4 : 7, 5 : 7, 7 : 6, 8 : 6, 9 : 11, 10 : 7, 13 : 7, 15 : 7, 16 : 7, 18 : 101, 
                   19 : 7, 20 : 7, 21 : 3, 22 : 5, 23 : 11, 25 : 11, 26 : 11, 30 : 7, 32 : 7, 34 : 98, 41 : 8, 52 : 2, 
                   53 : 6, 70 : 11}
wi_asgngrp_dict = {4 : 7, 5 : 6, 7 : 7, 8 : 11, 9 : 11, 10 : 1, 20 : 6, 26 : 6, 30 : 15, 36 : 2, 40 : 3, 41 : 7, 42 : 101,
                   43 : 11, 44 : 11, 45 : 7, 46 : 7, 99 : 11}

#rc_num corresp
mn_rc_num_dict = {1 : 10, 2 : 20, 3 : 30, 4 : 10, 5 : 20, 7 : 40, 8 : 50, 9 : 50, 10 : 30, 13 : 30, 15 : 30, 16 : 40,
                       18 : 101, 19 : 40, 20 : 50, 21 : 60, 22 : 40, 23 : 40, 25 : 50, 26 : 50, 30 : 50, 32 : 50, 34 : 101, 
                       41 : 10, 52 : 20, 53 : 30, 70 : 50}
wi_rc_num_dict = {4 : 20, 5 : 30, 7 : 40, 8 : 50, 9 : 50, 10 : 10, 20 : 20, 26 : 30, 30 : 20, 36 : 30, 40 : 60, 41 : 40, 
                  42 : 101, 43 : 50, 44 : 50, 45 : 50, 46 : 30, 99 : 50}

In [3]:
osm_drive_node_df = pd.read_csv(cd + "centroid_connector_builder/osm_drive_nodes_with_centroid_new.csv")[['N', 'OSMID']]
osm_walk_node_df = pd.read_csv(cd + "centroid_connector_builder/osm_walk_nodes_with_centroid_new.csv")[['N', 'OSMID']]

In [4]:
osm_drive_edge_gdf = gpd.read_file(osm_drive_edge_file)
osm_drive_edge_gdf = osm_drive_edge_gdf.to_crs({'init': 'epsg:26915', 'no_defs': True})

osm_walk_edge_gdf = gpd.read_file(osm_walk_edge_file)
osm_walk_edge_gdf = osm_walk_edge_gdf.to_crs({'init': 'epsg:26915', 'no_defs': True})

county_shape_gdf = gpd.read_file(county_shape_file)
county_shape_gdf = county_shape_gdf.to_crs(epsg = 26915)

area_gdf = gpd.read_file(area_file)[['COMDES2040', 'geometry']]
area_gdf = area_gdf.to_crs({'init': 'epsg:26915', 'no_defs': True})

In [7]:
mn_count_df = pd.read_csv(mn_count_file)[['id_count', 'CURR_VOL', 'LINK_ID']].drop_duplicates(['LINK_ID'])
wi_count_df = pd.read_csv(wi_count_file)[['id_count', 'RDWY_AADT', 'LINK_ID']].drop_duplicates(['LINK_ID'])

In [8]:
mn_fc_df = pd.read_csv(mn_fc_file)[['OBJECTID', 'ROUTE_SYS']]
wi_fc_df = pd.read_csv(wi_fc_file)[['OBJECTID', 'RDWY_CTGY_']]

fc_corresp_df = pd.read_csv(fc_corrsp_file)[['LINK_ID', 'mn_fc_id', 'wi_fc_id']]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3,6,7,8,9,11,12,20,21,22,23,24,29,30,31,32,35,36,49,51,52,53,54,60,61,62,66) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7,16,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
#map consecutive A, B node id
def map_AB_osmid_to_consecutive_id(osm_edge_gdf, osm_node_df):
    node_dict = dict(zip(osm_node_df.OSMID, osm_node_df.N))
    osm_edge_gdf['osmid_from'] = pd.to_numeric(osm_edge_gdf['osmid_from'], downcast = 'integer')
    osm_edge_gdf['osmid_to'] = pd.to_numeric(osm_edge_gdf['osmid_to'], downcast = 'integer')
    osm_edge_gdf['A'] = osm_edge_gdf['osmid_from'].map(node_dict)
    osm_edge_gdf['B'] = osm_edge_gdf['osmid_to'].map(node_dict)
    osm_edge_gdf.dropna(subset = ['A', 'B'], axis = 0, inplace = True)
    osm_edge_gdf['A'] = osm_edge_gdf['A'].astype(int)
    osm_edge_gdf['B'] = osm_edge_gdf['B'].astype(int)
    
    return osm_edge_gdf

In [12]:
#add county, area
def add_county_and_area_code(osm_edge_gdf, county_shape, area_shape):
    osm_edge_gdf['mid_point'] = osm_edge_gdf['geometry'].interpolate(0.5, normalized = True)
    osm_edge_gdf.rename(columns = {'mid_point' : 'geometry', 'geometry' : 'line_geom'}, inplace = True)

    osm_edge_gdf = gpd.sjoin(osm_edge_gdf, 
                        county_shape, 
                        how = 'left', 
                        op = 'intersects')
    osm_edge_gdf.drop('index_right', axis = 1, inplace = True)
    osm_edge_gdf = gpd.sjoin(osm_edge_gdf, 
                        area_shape, 
                        how = 'left', 
                        op = 'intersects')

    osm_edge_gdf.rename(columns = {'geometry' : 'mid_point', 'line_geom' : 'geometry'}, inplace = True)

    #add county
    osm_edge_gdf['COUNTY'] = osm_edge_gdf['NAME_right'].map(county_name_dict).fillna(10).astype(int)
    #add area
    osm_edge_gdf['AREA'] = osm_edge_gdf['COMDES2040'].map(area_code_dict).fillna(10).astype(int)
    
    return osm_edge_gdf

In [13]:
#add name
def add_road_name(osm_edge_gdf):
    osm_edge_gdf['NAME'] = osm_edge_gdf['NAME_left']
    
    return osm_edge_gdf

In [14]:
#add AADT
def add_counts(osm_edge_gdf, mn_count_df, wi_count_df):
    osm_edge_gdf = pd.merge(osm_edge_gdf, mn_count_df,
                       how = 'left',
                       on = 'LINK_ID')
    osm_edge_gdf = pd.merge(osm_edge_gdf, wi_count_df,
                       how = 'left',
                       on = 'LINK_ID')
    
    osm_edge_gdf['AADT'] = osm_edge_gdf[['CURR_VOL', 'RDWY_AADT']].max(axis = 1).fillna(0).astype(int)
    
    osm_edge_gdf['AM_CNT'] = (osm_edge_gdf['AADT'] / 4).astype(int)
    osm_edge_gdf['MD_CNT'] = (osm_edge_gdf['AADT'] / 4).astype(int)
    osm_edge_gdf['PM_CNT'] = (osm_edge_gdf['AADT'] / 4).astype(int)
    osm_edge_gdf['NT_CNT'] = (osm_edge_gdf['AADT'] / 4).astype(int)
    osm_edge_gdf['DY_CNT'] = osm_edge_gdf['AADT']
    
    return osm_edge_gdf

In [15]:
#add asgngrp, rc_num
def add_asgngrp_rc_num(osm_edge_gdf_in, mn_fc, wi_fc, fc_corresp):
    osm_edge_gdf = osm_edge_gdf_in.copy()
    osm_edge_gdf = pd.merge(osm_edge_gdf, fc_corresp,
                           how = 'left',
                           on = 'LINK_ID')
    osm_edge_gdf = pd.merge(osm_edge_gdf, mn_fc,
                           how = 'left',
                           left_on = 'mn_fc_id',
                           right_on = 'OBJECTID')
    osm_edge_gdf = pd.merge(osm_edge_gdf, wi_fc,
                           how = 'left',
                           left_on = 'wi_fc_id',
                           right_on = 'OBJECTID')
    osm_edge_gdf['mn_ASGN'] = osm_edge_gdf['ROUTE_SYS'].map(mn_asgngrp_dict).fillna(0).astype(int)
    osm_edge_gdf['wi_ASGN'] = osm_edge_gdf['RDWY_CTGY_'].map(wi_asgngrp_dict).fillna(0).astype(int)
    osm_edge_gdf['ASGNGRP'] = np.where(osm_edge_gdf['CENTROID'] == 0,
                                       osm_edge_gdf[['mn_ASGN', 'wi_ASGN']].max(axis = 1).astype(int),
                                       9)
    
    osm_edge_gdf['mn_RC'] = osm_edge_gdf['ROUTE_SYS'].map(mn_rc_num_dict).fillna(0).astype(int)
    osm_edge_gdf['wi_RC'] = osm_edge_gdf['RDWY_CTGY_'].map(wi_rc_num_dict).fillna(0).astype(int)
    osm_edge_gdf['RC_NUM'] = np.where(osm_edge_gdf['CENTROID'] == 0,
                                      osm_edge_gdf[['mn_RC', 'wi_RC']].max(axis = 1).astype(int),
                                      99)
    
    return osm_edge_gdf

In [16]:
#add lanes
def change_lane_name(osm_edge_gdf):
    osm_edge_gdf.rename(columns = {'LANES' :'osm_lanes', 'LANES_CONS' : 'LANES'}, inplace = True)
    osm_edge_gdf['LANES'] = osm_edge_gdf['LANES'].astype(int)
    
    return osm_edge_gdf

In [17]:
#Other Attributes
def add_other_attributes(osm_edge_gdf):
    osm_edge_gdf['T_PRIORITY'] = np.int(0)

    osm_edge_gdf['BIKE'] = np.where((osm_edge_gdf['RC_NUM'] < 30) | (osm_edge_gdf['RC_NUM'] == 60),
                                  0,
                                  np.where(osm_edge_gdf['RC_NUM'] == 101,
                                          1,
                                          3))
    osm_edge_gdf['HOV'] = np.where((osm_edge_gdf['ASGNGRP'] == 8) | (osm_edge_gdf['ASGNGRP'] == 10),
                                  100,
                                  0)
    return osm_edge_gdf

In [18]:
def remove_duplicate_a_b_links(osm_edge_gdf):
    osm_edge_gdf.drop_duplicates(['A', 'B'], inplace = True)
    
    return osm_edge_gdf

def remove_cul_de_sacs(osm_edge_gdf):
    osm_edge_gdf = osm_edge_gdf[osm_edge_gdf['A'] != osm_edge_gdf['B']]
    
    return osm_edge_gdf

In [19]:
def refresh_id(osm_edge_gdf):
    osm_edge_gdf['LINK_ID'] = range(1, 1+len(osm_edge_gdf))
    return osm_edge_gdf

In [20]:
def add_length(osm_edge_gdf):
    osm_edge_gdf['DISTANCE'] = osm_edge_gdf['geometry'].length/1609.34
    osm_edge_gdf['DISTANCE'] = osm_edge_gdf['DISTANCE'].round(10).astype(np.float16)
    return osm_edge_gdf

In [21]:
def build_roadway_network(osm_edge_gdf_input, osm_node_df_input, county_shape, area_shape, mn_count, wi_count, 
                          mn_fc, wi_fc, fc_corresp):
    osm_edge_gdf = osm_edge_gdf_input.copy()
    osm_edge_gdf = map_AB_osmid_to_consecutive_id(osm_edge_gdf, osm_node_df_input)
    osm_edge_gdf = add_county_and_area_code(osm_edge_gdf, county_shape, area_shape)
    osm_edge_gdf = add_road_name(osm_edge_gdf)
    osm_edge_gdf = add_counts(osm_edge_gdf, mn_count, wi_count)
    osm_edge_gdf = add_asgngrp_rc_num(osm_edge_gdf, mn_fc, wi_fc, fc_corresp)
    osm_edge_gdf = change_lane_name(osm_edge_gdf)
    osm_edge_gdf = add_other_attributes(osm_edge_gdf)
    osm_edge_gdf = remove_duplicate_a_b_links(osm_edge_gdf)
    osm_edge_gdf = remove_cul_de_sacs(osm_edge_gdf)
    osm_edge_gdf = refresh_id(osm_edge_gdf)
    osm_edge_gdf = add_length(osm_edge_gdf)
    
    column_list = ['A', 'B', 'LINK_ID', 'COUNTY', 'T_PRIORITY', 'AREA',
       'HOV', 'AADT', 'AM_CNT', 'MD_CNT', 'PM_CNT', 'NT_CNT', 'DY_CNT',
       'ASGNGRP', 'LANES', 'CENTROID', 'RC_NUM', 'geometry']
    
    osm_edge_gdf[column_list].to_file(cd + "/osm_roadway_network_builder/drive_link.shp")
    
    return osm_edge_gdf[column_list]

def build_walk_network(osm_edge_gdf_input, osm_node_df_input, county_shape, area_shape):
    osm_edge_gdf = osm_edge_gdf_input.copy()
    osm_edge_gdf = map_AB_osmid_to_consecutive_id(osm_edge_gdf, osm_node_df_input)
    osm_edge_gdf = add_county_and_area_code(osm_edge_gdf, county_shape, area_shape)
    osm_edge_gdf = add_road_name(osm_edge_gdf)
    osm_edge_gdf = remove_duplicate_a_b_links(osm_edge_gdf)
    osm_edge_gdf = remove_cul_de_sacs(osm_edge_gdf)
    osm_edge_gdf = refresh_id(osm_edge_gdf)
    osm_edge_gdf = add_length(osm_edge_gdf)
    
    column_list = ['A', 'B', 'LINK_ID', 'COUNTY', 'AREA', 'CENTROID', 'geometry']
    
    osm_edge_gdf[column_list].to_file(cd + "/osm_roadway_network_builder/walk_link.shp")
    
    return osm_edge_gdf[column_list]

In [27]:
def build_bike_network(build_drive_gdf):
    build_bike_gdf = build_drive_gdf.copy()
    #build_bike_gdf = build_bike_gdf[build_bike_gdf['BIKE'] > 0]
    build_bike_gdf['BIKE'] = np.int(3)
    build_bike_gdf['LINK_ID'] = range(1, 1+len(build_bike_gdf))
    
    column_list = ['A', 'B', 'LINK_ID', 'COUNTY', 'AREA', 'BIKE', 'CENTROID', 'geometry']
    
    build_bike_gdf[column_list].to_file(cd + "/osm_roadway_network_builder/bike_link.shp")
    
    return build_bike_gdf[column_list]

In [28]:
#build_osm_drive_edge_gdf = build_roadway_network(osm_drive_edge_gdf, osm_drive_node_df, county_shape_gdf, area_gdf, 
   #                                              mn_count_df, wi_count_df, mn_fc_df, wi_fc_df, fc_corresp_df)
#build_osm_walk_edge_gdf = build_walk_network(osm_walk_edge_gdf, osm_walk_node_df, county_shape_gdf, area_gdf)
build_osm_bike_edge_gdf = build_bike_network(build_osm_drive_edge_gdf)

In [29]:
(build_osm_drive_edge_gdf['A']==build_osm_drive_edge_gdf['B']).sum()

0

In [30]:
(build_osm_drive_edge_gdf.groupby(['A', 'B'])['LINK_ID'].count()>=2).sum()

0

In [32]:
def shp_float_fields(network_type):
    #additional step to add float fields
    source = ogr.Open(cd + "/osm_roadway_network_builder/" + network_type + "_link.shp", update=True)
    layer = source.GetLayer()
    layer_defn = layer.GetLayerDefn()
    field_names = [layer_defn.GetFieldDefn(i).GetName() for i in range(layer_defn.GetFieldCount())]
    print(len(field_names))

    #Add a new field - DISTANCE
    new_field = ogr.FieldDefn('DISTANCE', ogr.OFTReal)
    new_field.SetWidth(10)
    new_field.SetPrecision(5)
    if ('DISTANCE' in field_names) == False:
        layer.CreateField(new_field)

    #for i in layer:
    #    geom = i.GetGeometryRef()
    #    length = (geom.Length())/1609.344
    #    i.SetField( "DISTANCE", length)
    #    layer.SetFeature(i)

    #Add a new field - T_MANTIME
    if network_type == 'drive':
        new_field = ogr.FieldDefn('T_MANTIME', ogr.OFTReal)
        new_field.SetWidth(10) 
        new_field.SetPrecision(5)
        if ('T_MANTIME' in field_names) == False:
            layer.CreateField(new_field)

    for i in layer:
        geom = i.GetGeometryRef()
        length = (geom.Length())/1609.344
        i.SetField( "DISTANCE", length)
        if network_type == 'drive':
            i.SetField( "T_MANTIME", 0)
        layer.SetFeature(i)

    # Close the Shapefile
    source = None


for i in ['drive', 'walk', 'bike']:
    shp_float_fields(i)

17
6
7


In [38]:
build_osm_drive_edge_gdf.ASGNGRP.value_counts()

7     301777
6      87986
9      24180
3      10387
11      8507
5       4415
2       3254
15      2180
1       1561
0         12
8          7
Name: ASGNGRP, dtype: int64

In [ ]:
# save to csv
build_osm_drive_edge_gdf['isDriveLink'] = np.int(1)
build_osm_walk_edge_gdf['isPedLink'] = np.int(1)
build_osm_bike_edge_gdf['isBikeLink'] = np.int(1)

build_all_edge_gdf = pd.concat([build_osm_drive_edge_gdf, build_osm_walk_edge_gdf, build_osm_bike_edge_gdf], 
                               ignore_index=True, sort = False)

build_osm_drive_edge_gdf.to_csv(cd + "osm_roadway_network_builder/drive_network_for_modeling.csv", index = False)
build_osm_walk_edge_gdf.to_csv(cd + "osm_roadway_network_builder/walk_network_for_modeling.csv", index = False)
build_osm_bike_edge_gdf.to_csv(cd + "osm_roadway_network_builder/bike_network_for_modeling.csv", index = False)

build_all_edge_gdf.to_csv(cd + "osm_roadway_network_builder/network_for_modeling.csv", index = False)